### Load Packages

In [1]:
# Install Packages
%pip install pandas numpy matplotlib seaborn

# Import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import os

Note: you may need to restart the kernel to use updated packages.


### Load Datasets

In [2]:
# Load the MenuPage and Menu datasets
menupage_df = pd.read_csv('data/raw/MenuPage.csv')
menu_df = pd.read_csv('data/raw/Menu.csv')

print("MenuPage Dataset:")
display(menupage_df.head())

display(menu_df.head())

print("\nMenuPage Data Types:")
print(menupage_df.dtypes)

print("\nMenuPage Summary Statistics:")
display(menupage_df.describe())

MenuPage Dataset:


,id,menu_id,page_number,image_id,full_height,full_width,uuid
0,119,12460,1.0,1603595,7230.0,5428.0,510d47e4-2955-a3d9-e040-e00a18064a99
1,120,12460,2.0,1603596,5428.0,7230.0,510d47e4-2956-a3d9-e040-e00a18064a99
2,121,12460,3.0,1603597,7230.0,5428.0,510d47e4-2957-a3d9-e040-e00a18064a99
3,122,12460,4.0,1603598,7230.0,5428.0,510d47e4-2958-a3d9-e040-e00a18064a99
4,123,12461,1.0,1603591,7230.0,5428.0,510d47e4-2959-a3d9-e040-e00a18064a99


,id,name,sponsor,event,venue,place,physical_description,occasion,notes,call_number,keywords,language,date,location,location_type,currency,currency_symbol,status,page_count,dish_count
0,12463,NaN,HOTEL EASTMAN,BREAKFAST,COMMERCIAL,"HOT SPRINGS, AR",CARD; 4.75X7.5;,EASTER;,NaN,1900-2822,NaN,NaN,1900-04-15,Hotel Eastman,NaN,NaN,NaN,complete,2,67
1,12464,NaN,REPUBLICAN HOUSE,[DINNER],COMMERCIAL,"MILWAUKEE, [WI];",CARD; ILLUS; COL; 7.0X9.0;,EASTER;,WEDGEWOOD BLUE CARD; WHITE EMBOSSED GREEK KEY ...,1900-2825,NaN,NaN,1900-04-15,Republican House,NaN,NaN,NaN,under review,2,34
2,12465,NaN,NORDDEUTSCHER LLOYD BREMEN,FRUHSTUCK/BREAKFAST;,COMMERCIAL,DAMPFER KAISER WILHELM DER GROSSE;,CARD; ILLU; COL; 5.5X8.0;,NaN,"MENU IN GERMAN AND ENGLISH; ILLUS, STEAMSHIP A...",1900-2827,NaN,NaN,1900-04-16,Norddeutscher Lloyd Bremen,NaN,NaN,NaN,complete,2,84
3,12466,NaN,NORDDEUTSCHER LLOYD BREMEN,LUNCH;,COMMERCIAL,DAMPFER KAISER WILHELM DER GROSSE;,CARD; ILLU; COL; 5.5X8.0;,NaN,"MENU IN GERMAN AND ENGLISH; ILLUS, HARBOR SCEN...",1900-2828,NaN,NaN,1900-04-16,Norddeutscher Lloyd Bremen,NaN,NaN,NaN,complete,2,63
4,12467,NaN,NORDDEUTSCHER LLOYD BREMEN,DINNER;,COMMERCIAL,DAMPFER KAISER WILHELM DER GROSSE;,FOLDER; ILLU; COL; 5.5X7.5;,NaN,"MENU IN GERMAN AND ENGLISH; ILLUS, HARBOR SCEN...",1900-2829,NaN,NaN,1900-04-16,Norddeutscher Lloyd Bremen,NaN,NaN,NaN,complete,4,33



MenuPage Data Types:
id               int64
menu_id          int64
page_number    float64
image_id        object
full_height    float64
full_width     float64
uuid            object
dtype: object

MenuPage Summary Statistics:


,id,menu_id,page_number,full_height,full_width
count,66937.000000,66937.000000,65735.000000,66608.000000,66608.000000
mean,42719.760596,25653.580904,3.759169,3859.102856,2778.588398
std,21273.997831,6158.827683,4.909050,1156.010307,970.287300
min,119.000000,12460.000000,1.000000,616.000000,558.000000
25%,27108.000000,21743.000000,1.000000,2988.000000,2120.000000
50%,43894.000000,26202.000000,2.000000,3630.000000,2527.000000
75%,60696.000000,30531.000000,4.000000,4617.250000,3295.250000
max,77431.000000,35526.000000,74.000000,12044.000000,9175.000000


#### Analyze IC Violations Between MenuPage and Menu Datasets
##### Section 1

In [3]:
# Find MenuPage records that don't link back to valid Menu IDs
menupage_menu_ids = set(menupage_df['menu_id'])
valid_menu_ids = set(menu_df['id'])
violating_menu_ids = menupage_menu_ids - valid_menu_ids

print(f"Total unique menu IDs in MenuPage: {len(menupage_menu_ids)}")
print(f"Total valid menu IDs in Menu table: {len(valid_menu_ids)}")
print(f"MenuPage records with invalid menu IDs: {len(violating_menu_ids)}")

# Count MenuPage records that violate the IC constraint
violating_records = menupage_df[menupage_df['menu_id'].isin(violating_menu_ids)]
total_menupage_records = len(menupage_df)
violation_percentage = (len(violating_records) / total_menupage_records) * 100

print(f"Total MenuPage records: {total_menupage_records}")
print(f"Records with invalid menu_id: {len(violating_records)}")
print(f"Violation percentage: {violation_percentage}%")

Total unique menu IDs in MenuPage: 19816
Total valid menu IDs in Menu table: 17545
MenuPage records with invalid menu IDs: 2271
Total MenuPage records: 66937
Records with invalid menu_id: 5803
Violation percentage: 8.669345802769769%


### Page Number Validation
#### Section 2

In [4]:
total_records = len(menupage_df)

# Check for null values in page_number
null_page_numbers = menupage_df['page_number'].isnull().sum()
null_percentage = (null_page_numbers / total_records) * 100

print(f"Null page_number values: {null_page_numbers} ({null_percentage}%)")

# Check for negative values in page_number  
negative_page_numbers = (menupage_df['page_number'] < 0).sum()
negative_percentage = (negative_page_numbers / total_records) * 100

print(f"Negative page_number values: {negative_page_numbers} ({negative_percentage}%)")

# Check for zero values in page_number - would expect the page number to start at 1
zero_page_numbers = (menupage_df['page_number'] == 0).sum()
zero_percentage = (zero_page_numbers / total_records) * 100

print(f"Zero page_number values: {zero_page_numbers} ({zero_percentage}%)")


Null page_number values: 1202 (1.79571836204192%)
Negative page_number values: 0 (0.0%)
Zero page_number values: 0 (0.0%)
